In [ ]:
using Random
using Printf
using DelimitedFiles
using Statistics
using LinearAlgebra
using LaTeXStrings
using PyCall, PyPlot
plt = pyimport("matplotlib.pyplot")

include("WFD/WFD.jl")
include("WFD/common.jl");

# Demonstration of the first recovery result

In [ ]:
# Fig A - sparse graph good attributes

edges = readdlm("data/synthetic/sbm_sparse.txt", ' ', Int, '\n')
n = 10000
ll = [Vector{Int64}() for _ in 1:n]
for e in eachrow(edges)
    push!(ll[e[1]], e[2])
    push!(ll[e[2]], e[1])
end
degree = [length(l) for l in ll]
G = AdjacencyList(ll, degree, n)

k = 500
sigma = 1
d = 100
mu = 1.5*log(n)/sqrt(d)

Z = zeros(n,1)
X = zeros(n,d)
X[1:k,:] .= mu
X[(k+1):end,:] .= -mu
X += randn(n,d)
gamma = 1/4/log(n)^(3/2)
edge_weight_z = edge_weights(G, Z, 0.)
edge_weight_x = edge_weights(G, X, gamma)

precision_X = zeros(50,50)
precision_Z = zeros(50,50)
recall_X = zeros(50,50)
recall_Z = zeros(50,50)

sink = ones(Float64, G.nv)

for (i,alpha) in enumerate(range(0.1, 5, length=50))
    println("alpha = ", alpha)
    for s in 1:50
        source = zeros(G.nv)
        source[s] = alpha*k
        x = WFD(G, source, sink, weights=edge_weight_x, max_iters=1000, epsilon=1.0e-4)
        C = findall(!iszero,x)
        precision_X[i,s] = count(C.<=500)/length(C)
        recall_X[i,s] = count(C.<=500)/k
        x = WFD(G, source, sink, weights=edge_weight_z, max_iters=1000, epsilon=1.0e-4)
        C = findall(!iszero,x)
        precision_Z[i,s] = count(C.<=500)/length(C)
        recall_Z[i,s] = count(C.<=500)/k
    end
end

precision_X_mean = vec(mean(precision_X, dims=2))
precision_X_std = vec(std(precision_X, dims=2))
precision_Z_mean = vec(mean(precision_Z, dims=2))
precision_Z_std = vec(std(precision_Z, dims=2))
recall_X_mean = vec(mean(recall_X, dims=2))
recall_X_std = vec(std(recall_X, dims=2))
recall_Z_mean = vec(mean(recall_Z, dims=2))
recall_Z_std = vec(std(recall_Z, dims=2))

fig, ax = plt.subplots()
plt.plot(range(0.1,5,length=50), recall_X_mean, marker="s", markersize=7, 
    color="blue", linewidth=2, label="Recall")
plt.fill_between(range(0.1,5,length=50), recall_X_mean-recall_X_std, 
    recall_X_mean+recall_X_std, alpha=.2, color="blue")
plt.plot(range(0.1,5,length=50), precision_X_mean, marker="d", markersize=7, 
    color="tab:green", linewidth=2, label="Precision")
plt.fill_between(range(0.1,5,length=50), precision_X_mean-precision_X_std, 
    precision_X_mean+precision_X_std, alpha=.2, color="tab:green")
plt.axvline(x = 1.1, color = "gray", linewidth=2,label=L"$\alpha=1.1$")

plt.plot(range(0.1,5,length=50), recall_Z_mean, linestyle="dotted", 
    color="steelblue", linewidth=3, label="Recall (no attributes)")
plt.fill_between(range(0.1,5,length=50), recall_Z_mean-recall_Z_std, 
    recall_Z_mean+recall_Z_std, alpha=.2, color="steelblue")
plt.plot(range(0.1,5,length=50), precision_Z_mean, linestyle="dashed", 
    color="darkseagreen", linewidth=3, label="Precision (no attributes)")
plt.fill_between(range(0.1,5,length=50), precision_Z_mean-precision_Z_std, 
    precision_Z_mean+precision_Z_std, alpha=.2, color="darkseagreen")

plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel(L"$\alpha$", size=18)
plt.legend(fontsize=15, loc = 7, bbox_to_anchor=(1, 0.38))
plt.savefig("vary_alpha_sparse_graph_good_attributes.pdf", bbox_inches="tight", format="pdf")

In [ ]:
# Fig B - sparse graph okay attributes

edges = readdlm("data/synthetic/sbm_sparse.txt", ' ', Int, '\n')
n = 10000
ll = [Vector{Int64}() for _ in 1:n]
for e in eachrow(edges)
    push!(ll[e[1]], e[2])
    push!(ll[e[2]], e[1])
end
degree = [length(l) for l in ll]
G = AdjacencyList(ll, degree, n)

k = 500
sigma = 1
d = 100
mu = 1.25*log(n)/sqrt(d)

Z = zeros(n,1)
X = zeros(n,d)
X[1:k,:] .= mu
X[(k+1):end,:] .= -mu
X += randn(n,d)
gamma = 1/4/log(n)^(3/2)
edge_weight_z = edge_weights(G, Z, 0.)
edge_weight_x = edge_weights(G, X, gamma)

precision_X = zeros(50,50)
precision_Z = zeros(50,50)
recall_X = zeros(50,50)
recall_Z = zeros(50,50)

sink = ones(Float64, G.nv)

for (i,alpha) in enumerate(range(0.1, 5, length=50))
    println("alpha = ", alpha)
    for s in 1:50
        source = zeros(G.nv)
        source[s] = alpha*k
        x = WFD(G, source, sink, weights=edge_weight_x, max_iters=1000, epsilon=1.0e-4)
        C = findall(!iszero,x)
        precision_X[i,s] = count(C.<=500)/length(C)
        recall_X[i,s] = count(C.<=500)/k
        x = WFD(G, source, sink, weights=edge_weight_z, max_iters=1000, epsilon=1.0e-4)
        C = findall(!iszero,x)
        precision_Z[i,s] = count(C.<=500)/length(C)
        recall_Z[i,s] = count(C.<=500)/k
    end
end

precision_X_mean = vec(mean(precision_X, dims=2))
precision_X_std = vec(std(precision_X, dims=2))
precision_Z_mean = vec(mean(precision_Z, dims=2))
precision_Z_std = vec(std(precision_Z, dims=2))
recall_X_mean = vec(mean(recall_X, dims=2))
recall_X_std = vec(std(recall_X, dims=2))
recall_Z_mean = vec(mean(recall_Z, dims=2))
recall_Z_std = vec(std(recall_Z, dims=2))

fig, ax = plt.subplots()
plt.plot(range(0.1,5,length=50), recall_X_mean, marker="s", markersize=7, 
    color="blue", linewidth=2, label="Recall")
plt.fill_between(range(0.1,5,length=50), recall_X_mean-recall_X_std, 
    recall_X_mean+recall_X_std, alpha=.2, color="blue")
plt.plot(range(0.1,5,length=50), precision_X_mean, marker="d", markersize=7, 
    color="tab:green", linewidth=2, label="Precision")
plt.fill_between(range(0.1,5,length=50), precision_X_mean-precision_X_std, 
    precision_X_mean+precision_X_std, alpha=.2, color="tab:green")
plt.axvline(x = 1.1, color = "gray", linewidth=2,label=L"$\alpha=1.1$")


plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel(L"$\alpha$", size=18)
plt.savefig("vary_alpha_sparse_graph_okay_attributes.pdf", bbox_inches="tight", format="pdf")

In [ ]:
# Fig C - denser graph okay attributes

edges = readdlm("data/synthetic/sbm.txt", ' ', Int, '\n')
n = 10000
ll = [Vector{Int64}() for _ in 1:n]
for e in eachrow(edges)
    push!(ll[e[1]], e[2])
    push!(ll[e[2]], e[1])
end
degree = [length(l) for l in ll]
G = AdjacencyList(ll, degree, n)

k = 500
sigma = 1
d = 100
mu = 1.25*log(n)/sqrt(d)

Z = zeros(n,1)
X = zeros(n,d)
X[1:k,:] .= mu
X[(k+1):end,:] .= -mu
X += randn(n,d)
gamma = 1/4/log(n)^(3/2)
edge_weight_z = edge_weights(G, Z, 0.)
edge_weight_x = edge_weights(G, X, gamma)

precision_X = zeros(50,50)
precision_Z = zeros(50,50)
recall_X = zeros(50,50)
recall_Z = zeros(50,50)

sink = ones(Float64, G.nv)

for (i,alpha) in enumerate(range(0.1, 5, length=50))
    println("alpha = ", alpha)
    for s in 1:50
        source = zeros(G.nv)
        source[s] = alpha*k
        x = WFD(G, source, sink, weights=edge_weight_x, max_iters=1000, epsilon=1.0e-4)
        C = findall(!iszero,x)
        precision_X[i,s] = count(C.<=500)/length(C)
        recall_X[i,s] = count(C.<=500)/k
        x = WFD(G, source, sink, weights=edge_weight_z, max_iters=1000, epsilon=1.0e-4)
        C = findall(!iszero,x)
        precision_Z[i,s] = count(C.<=500)/length(C)
        recall_Z[i,s] = count(C.<=500)/k
    end
end

precision_X_mean = vec(mean(precision_X, dims=2))
precision_X_std = vec(std(precision_X, dims=2))
precision_Z_mean = vec(mean(precision_Z, dims=2))
precision_Z_std = vec(std(precision_Z, dims=2))
recall_X_mean = vec(mean(recall_X, dims=2))
recall_X_std = vec(std(recall_X, dims=2))
recall_Z_mean = vec(mean(recall_Z, dims=2))
recall_Z_std = vec(std(recall_Z, dims=2))

fig, ax = plt.subplots()
plt.plot(range(0.1,5,length=50), recall_X_mean, marker="s", markersize=7, 
    color="blue", linewidth=2, label="Recall")
plt.fill_between(range(0.1,5,length=50), recall_X_mean-recall_X_std, 
    recall_X_mean+recall_X_std, alpha=.2, color="blue")
plt.plot(range(0.1,5,length=50), precision_X_mean, marker="d", markersize=7, 
    color="tab:green", linewidth=2, label="Precision")
plt.fill_between(range(0.1,5,length=50), precision_X_mean-precision_X_std, 
    precision_X_mean+precision_X_std, alpha=.2, color="tab:green")
plt.axvline(x = 1.1, color = "gray", linewidth=2,label=L"$\alpha=1.1$")

plt.plot(range(0.1,5,length=50), recall_Z_mean, linestyle="dotted", 
    color="steelblue", linewidth=3, label="Recall (no attributes)")
plt.fill_between(range(0.1,5,length=50), recall_Z_mean-recall_Z_std, 
    recall_Z_mean+recall_Z_std, alpha=.2, color="steelblue")
plt.plot(range(0.1,5,length=50), precision_Z_mean, linestyle="dashed", 
    color="darkseagreen", linewidth=3, label="Precision (no attributes)")
plt.fill_between(range(0.1,5,length=50), precision_Z_mean-precision_Z_std, 
    precision_Z_mean+precision_Z_std, alpha=.2, color="darkseagreen")

plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel(L"$\alpha$", size=18)
plt.savefig("vary_alpha_denser_graph_okay_attributes.pdf", bbox_inches="tight", format="pdf")

# Demonstration of the second recovery result

In [ ]:
edges = readdlm("data/synthetic/sbm.txt", ' ', Int, '\n')
n = 10000
ll = [Vector{Int64}() for _ in 1:n]
for e in eachrow(edges)
    push!(ll[e[1]], e[2])
    push!(ll[e[2]], e[1])
end
degree = [length(l) for l in ll]
G = AdjacencyList(ll, degree, n)

k = 500
sigma = 1
d = 100
Z = zeros(n,1)
noise = randn(n,d)
gamma = 1/4/log(n)^(3/2)

BEST_F1 = zeros(21,50)
SELECTED_F1 = zeros(21,50)

for (i,a) in enumerate(0:0.2:4)
    
    @printf("a = %.1f\n", a)
    flush(stdout)
    
    mu = a*sqrt(log(n))/sqrt(d)
    X = zeros(n,d)
    X[1:k,:] .= mu
    X[(k+1):end,:] .= -mu
    X += noise
    
    edge_weight = edge_weights(G, X, gamma)
    
    for s in 1:50
        
        best_f1 = 0
        best_cond = 1
        selected_f1 = 0
        
        for alpha = 1.1:0.5:10.1  
            source = zeros(n)
            source[s] = alpha*k
            sink = ones(n)
            x = WFD(G, source, sink, weights=edge_weight, max_iters=100, epsilon=1.0e-4)
            C = findall(!iszero,x)
            pr = count(C.<=500)/length(C)
            re = count(C.<=500)/k
            f1 = 2*pr*re/(pr+re)
            if f1 > best_f1
                best_f1 = f1
            end
            _, _, conductance = compute_conductance(G, C, edge_weight=edge_weight)
            if conductance < best_cond
                best_cond = conductance
                selected_f1 = f1
            end        
        end
        
        BEST_F1[i,s] = best_f1
        SELECTED_F1[i,s] = selected_f1
        
    end       
end

BASE_BEST_F1 = zeros(50)
BASE_SELECTED_F1 = zeros(50)

edge_weight = edge_weights(G, Z, 0.)

for s in 1:50
        
    best_f1 = 0
    best_cond = 1
    selected_f1 = 0
        
    for alpha = 1.1:0.5:10.1
        
        source = zeros(n)
        source[s] = alpha*k
        sink = ones(n)
        x = WFD(G, source, sink, weights=edge_weight, max_iters=100, epsilon=1.0e-4)
        C = findall(!iszero,x)
        pr = count(C.<=500)/length(C)
        re = count(C.<=500)/k
        f1 = 2*pr*re/(pr+re)
        if f1 > best_f1
            best_f1 = f1
        end
        _, _, conductance = compute_conductance(G, C, weighted=false)
        if conductance < best_cond
            best_cond = conductance
            selected_f1 = f1
        end
      
    end
        
    BASE_BEST_F1[s] = best_f1
    BASE_SELECTED_F1[s] = selected_f1
    
end

base_best_f1_mean = mean(BASE_BEST_F1)
base_best_f1_std = std(BASE_BEST_F1)
best_f1_mean = vec(mean(BEST_F1, dims=2))
best_f1_std = vec(std(BEST_F1, dims=2))
selected_f1_mean = vec(mean(SELECTED_F1, dims=2))
selected_f1_std = vec(std(SELECTED_F1, dims=2))

fig, ax = plt.subplots(figsize=(8, 5))
plt.plot(collect(0:0.4:8), best_f1_mean, marker="o", markersize=8, 
    color="red", linewidth=2, label="Best performance")
plt.fill_between(collect(0:0.4:8), best_f1_mean-best_f1_std, 
    best_f1_mean+best_f1_std, alpha=.2, color="red")
plt.plot(collect(0:0.4:8), selected_f1_mean, marker="o", markersize=8, 
    color="blue", linewidth=2, label="Perf. for min conductance")
plt.fill_between(collect(0:0.4:8), selected_f1_mean-selected_f1_std, 
    selected_f1_mean+selected_f1_std, alpha=.2, color="blue")
plt.plot(collect(0:0.4:8), [base_best_f1_mean for _ in 1:length(collect(0:0.4:8))],
    color="black", linewidth=3, linestyle="dotted", label="Best perf. without attributes")
plt.fill_between(collect(0:0.4:8), [base_best_f1_mean-base_best_f1_std for _ in 1:length(collect(0:0.4:8))],
    [base_best_f1_mean+base_best_f1_std for _ in 1:length(collect(0:0.4:8))], alpha=.2, color="black")
plt.axvline(x = 1, color = "g", linewidth=2,label=L"$\hat\mu = \hat\sigma\sqrt{\log n}$")
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Strength of node attributes", size=16)
plt.ylabel("F1 score", size=16)
plt.legend(fontsize=14, loc = 2)
plt.savefig("vary_mu_denser_graph.pdf", bbox_inches="tight", format="pdf")